In [76]:
import pandas as pd
import ast
import pytz
import os
import json
import yaml
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS
from dateutil import tz
from datetime import datetime
from datetime import timedelta
from IPython.display import display

pd.set_option('display.max_columns', None)

class BasePipeline:
    def __init__(self, config, base_file_path):
        self.config = config
        self.base_file_path = base_file_path
        self.geomap = {}
        self._data = pd.DataFrame({})
    
    def construct_file_path(self):
        #TODO: add type to prefix mapping
        file_prefix = 'per_outage' if self.config['type'] == 'o' else 'per_county'
        file_path = f"{self.base_file_path}/{self.config['state']}/layout_{self.config['layout']}/{file_prefix}_{self.config['name']}.csv"
        return file_path.replace('//', '/')

    def load_data(self):
        # TODO: use us zipcode database
        try:
            file_path = self.construct_file_path()
            # print(file_path)
            self._data = pd.read_csv(file_path)
            with open('zip_to_county_name.json', 'r') as json_file:
                self.geomap['zip_to_county_name'] = json.load(json_file)
            with open('zip_to_county_fips.json', 'r') as json_file:
                self.geomap['zip_to_county_fips'] = json.load(json_file)
            with open('zip_to_state_name.json', 'r') as json_file:
                self.geomap['zip_to_state_name'] = json.load(json_file)
        except Exception as e:
            print(f"An error occurred during file loading: {e}")
            
    def transform(self):
        # Base transformation method
        raise NotImplementedError

    def standardize(self):
        # Base transformation method
        # where most of the functionality is
        self.load_data()
        self.transform()
        grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)
        self._data = pd.merge(grouped, self._data, on=['outage_id', 'timestamp'], how='inner')
        
        self._data['utility_provider'] = self.config['name']
        self._data['state'] = self.config['state']
        self._data['county'] = self._data['zipcode'].map(self.geomap)
        
        self._data = self._data[[
            'utility_provider', 'state', 'county', 'zipcode',
            'outage_id', 'start_time', 'end_time', 'lat', 'lng', 
            'duration', 'duration_max', 'duration_mean', 'customer_affected_mean', 'total_customer_outage_time', 'total_customer_outage_time_max', 'total_customer_outage_time_mean'
        ]]
        
        return self._data
    
    def output_data(self, standard_data):
        # TODO: Output unified data
        pass
    
    def get_dataframe(self):
        return self._data
    
    def _compute_metrics(self, group):
        duration = (group['end_time'] - group['start_time']).dt.total_seconds() / 60
        duration_max = duration + 15
        duration_mean = (duration + duration_max) / 2
        customer_affected_mean = group['customer_affected'].mean()
        
        total_customer_outage_time = 15 * (group['customer_affected'].sum() - group['customer_affected'].iloc[0]) + (group['timestamp'].iloc[0] - group['start_time'].iloc[0]).total_seconds() / 60 * group['customer_affected'].iloc[0]
        total_customer_outage_time_max = total_customer_outage_time + 15 * group['customer_affected'].iloc[-1]
        total_customer_outage_time_mean = (total_customer_outage_time + total_customer_outage_time_max) / 2

        return pd.Series({
            'timestamp': group['end_time'].iloc[-1],
            'duration': duration.iloc[-1],
            'duration_max': duration_max.iloc[-1],
            'duration_mean': duration_mean.iloc[-1],
            'customer_affected_mean': customer_affected_mean,
            'total_customer_outage_time': total_customer_outage_time,
            'total_customer_outage_time_max': total_customer_outage_time_max,
            'total_customer_outage_time_mean': total_customer_outage_time_mean
        })
        
    def _check_other_vars(self):
        # TODO: Check other useful variables
        pass

class GA2TX17(BasePipeline):
    def transform(self):
        # helper function to truncate the seconds value of the timestamp
        def reformat_starttime(startTime): 
            # format: 2023-03-15T21:51:54-04:00
            reformatted = startTime
            if pd.notna(startTime) and "." in startTime: # if not NaN and there is a decimal (assuming only seconds can have decimals)
                # Find the index of the first period
                first_period_index = startTime.find('.')

                # Find the index of the first hyphen after the first period
                first_hyphen_after_period_index = startTime.find('-', first_period_index)
                
                # essentially removing the decimal part of the seconds 
                reformatted = startTime[:first_period_index] + startTime[first_hyphen_after_period_index:]

            return reformatted
        
        try:
            # Convert timestamps
            eastern = tz.gettz('US/Eastern')
            utc = tz.gettz('UTC')
            # truncated_time = dataframe['timestamp'].str[:19]
            self._data['OutageStartTime'] = self._data['OutageStartTime'].apply(reformat_starttime)
            self._data['timestamp'] = pd.to_datetime(self._data['timestamp'], utc=True).dt.tz_convert(eastern)
            self._data['OutageStartTime'] = pd.to_datetime(self._data['OutageStartTime'], format='mixed', utc=True).dt.tz_convert(eastern)
            self._data['OutageEndTime'] =pd.to_datetime(self._data['OutageEndTime'], format='mixed', utc=True).dt.tz_convert(eastern)

            # extract lat and long
            self._data['OutageLocation'] = self._data['OutageLocation'].apply(lambda x: json.loads(x.replace("'", '"')))
            self._data[['lat', 'long']] = self._data['OutageLocation'].apply(lambda x: pd.Series([x['Y'], x['X']]))
            
            self._data.rename(columns={
                'OutageRecID':'outage_id',
                'OutageStartTime': 'start_time',
                'CustomersOutNow':'customer_affected',
                'EMC': 'utility_provider',
                'zip': 'zipcode'
            }, inplace=True)
        except Exception as e:
            print(f"An error occurred during transformation: {e}")

    def standardize(self): # outage_data is another argument in the original
        self.load_data()
        self.transform()
        grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)
        self._data = grouped

    
    # gis = GIS("http://www.arcgis.com", "JK9035", "60129@GR0W3R5") # signing in to get access to arcGIS api
    def _compute_metrics(self, group):
        # helper method to get the zipcode from the latitudes and longitudes of each group
        def get_zipcode(long, lat): # using arcgis package
            location = reverse_geocode((Geometry({"x":float(long), "y":float(lat), "spatialReference":{"wkid": 4326}})))
            return location['address']['Postal']
        
        start_time = group['start_time'].min()
        duration_diff = group['timestamp'].max() - group['timestamp'].min()
        end_time = start_time + duration_diff
        lat = group['lat'].iloc[-1]
        long = group['long'].iloc[-1]
        # zipcode = get_zipcode(long, lat)

        zipcode = '000000' # dummy value to avoid using reverse_geocode since virtually all of the zip's provided are null
        county_name = pd.NA
        county_fips = pd.NA
        state = pd.NA
        utility_provider = group['utility_provider'].iloc[-1]
        duration_max = duration_diff + timedelta(minutes=15)
        duration_mean = (duration_diff + duration_max) / 2
        customer_affected_mean = group['customer_affected'].mean()
        total_customer_outage_time = customer_affected_mean * duration_diff

        return pd.Series({
            'start_time': start_time,
            'end_time': end_time,
            'lat': lat,
            'long': long,
            'zipcode': zipcode,
            'county_name': county_name,
            'county_fips': county_fips,
            'state': state,
            'utility_provider': utility_provider,
            'duration_max': duration_max,
            'duration_mean': duration_mean,
            'customer_affected_mean': customer_affected_mean,
            'total_customer_outage_time': total_customer_outage_time
        })

Establishing config files

In [50]:
local_config_path = '/Users/uirja/OneDrive/Personal Files/CS Projects/outage-data-scraper/app/pipeline/config.yaml'
with open(local_config_path, 'r') as file:
    config = yaml.safe_load(file)
    base_file_path = config['globals']['local_base_file_path']

# Test code to feel out the datasets


Loading configs manually and separtely


In [51]:
config_list = [
    (blue_conf := {'name': 'Blueridge Mountain EMC', 'state': 'ga', 'layout': 2, 'type': 'o'}),
    (tric_conf := {'name': 'Tri-County EMC', 'state': 'ga', 'layout': 2, 'type': 'o'}),
    (united_conf := {'name': 'United Coop Services, Inc.', 'state': 'tx', 'layout': 17, 'type': 'o'})
]

pipelines = [
    (blue_pipeline := GA2TX17(blue_conf, base_file_path)),
    (tric_pipeline := GA2TX17(tric_conf, base_file_path)),
    (united_pipeline := GA2TX17(united_conf, base_file_path))
]

for pipeline in pipelines:
    pipeline.load_data()


dataframes = [
    (blue_df := blue_pipeline._data),
    (tric_df := tric_pipeline._data),
    (united_df := united_pipeline._data)
]

# display(blue_df)
display(tric_df)
# display(united_df)

C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\2338607318.py:35: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  self._data = pd.read_csv(file_path)
C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\2338607318.py:35: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  self._data = pd.read_csv(file_path)


,OutageLocation,OutagePolygon,OutageRecID,OutageName,OutageStartTime,OutageEndTime,CustomersRestored,CustomersOutNow,Verified,CrewDispatched,CustomerResponsible,EstimatedTime,TroubledElement,Substation,Feeder,OutagedPhase,Actions,Status,Dispatcher,ClosingDispatcher,CustomersOutInitially,CustomersServed,SuspectedCause,VerifiedCause,Notes,CrewResponsible,MapLocation,PriorityCustomers,District,UplineElement,CostEstimate,oID,zip,EMC,timestamp
0,"{'X': -83.39671001640346, 'Y': 33.20068745977006}","[{'X': -83.39677793305296, 'Y': 33.20058896338...",2023-03-18-0055,35D-6-1,2023-03-18T15:18:04-04:00,NaN,0,2,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-18-0055,unknown,Tri-Countt EMC,NaN
1,"{'X': -83.47158406818582, 'Y': 32.92295549882344}","[{'X': -83.47176912626755, 'Y': 32.92254168802...",2023-04-14-0137,13-27-1,2023-04-14T13:23:00-04:00,NaN,0,2,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-14-0137,NaN,Tri-Countt EMC,04-14-2023 17:39:40
2,"{'X': -83.47229808319233, 'Y': 32.92224148381695}","[{'X': -83.47248314127404, 'Y': 32.92182767301...",2023-04-14-0137,13-27-1,2023-04-14T13:23:00-04:00,NaN,0,2,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-14-0137,NaN,Tri-Countt EMC,04-14-2023 17:54:40
3,"{'X': -83.47866346825026, 'Y': 32.915876098759...","[{'X': -83.47884852633197, 'Y': 32.91546228795...",2023-04-14-0137,13-27-1,2023-04-14T13:23:00-04:00,NaN,0,2,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-14-0137,NaN,Tri-Countt EMC,04-14-2023 18:09:40
4,"{'X': -83.49619713340982, 'Y': 32.89834243359947}","[{'X': -83.49638219149153, 'Y': 32.89792862279...",2023-04-14-0137,13-27-1,2023-04-14T13:23:00-04:00,NaN,0,2,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-14-0137,NaN,Tri-Countt EMC,04-14-2023 18:24:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14154,"{'X': -83.49337250235277, 'Y': 33.3536632913507}",[],2024-01-17-0387,12G-6-1,2024-01-17T19:22:48-05:00,NaN,0,1,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-17-0387,NaN,Tri-Countt EMC,01-18-2024 01:24:39
14155,"{'X': -83.51333738253444, 'Y': 33.34180853624282}",[],2024-01-17-0387,12G-6-1,2024-01-17T19:22:48-05:00,NaN,0,1,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-17-0387,NaN,Tri-Countt EMC,01-18-2024 01:39:39
14156,"{'X': -83.39599612471932, 'Y': 33.20976864531068}","[{'X': -83.39649375298593, 'Y': 33.20975573834...",2024-01-18-0388,35G-7,2024-01-18T07:58:47-05:00,NaN,0,3,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-18-0388,NaN,Tri-Countt EMC,01-18-2024 13:09:40
14157,"{'X': -83.3835229346058, 'Y': 33.22224183542419}","[{'X': -83.38402056287242, 'Y': 33.22222892845...",2024-01-18-0388,35G-7,2024-01-18T07:58:47-05:00,NaN,0,3,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-18-0388,NaN,Tri-Countt EMC,01-18-2024 13:24:39


In [52]:
for df in dataframes:
    print(f"Num of columns in {df.EMC.iloc[0]}: {len(df.columns)}")


Num of columns in Blueridge Mountain EMC: 36
Num of columns in Tri-Countt EMC: 35
Num of columns in United Coop Services, Inc.: 34


We notice that the three datasets do not have the same length in their columns, so we will have to see the unique column differences between them.

In [53]:
# get the set of column names for each dataframe
columns_blue = set(blue_df.columns)
columns_tric = set(tric_df.columns)
columns_united = set(united_df.columns)

# Find the symmetric differences between the sets of column names
diff_blue_tric = columns_blue.symmetric_difference(columns_tric)
diff_blue_united = columns_blue.symmetric_difference(columns_united)
diff_tric_united = columns_tric.symmetric_difference(columns_united)


# Print the differences
print("Columns that Blueridge and Tri-County don't share:", diff_blue_tric)
print("Columns that Blueridge and United don't share:", diff_blue_united)
print("Columns that Tri-County and United don't share:", diff_tric_united) # zero-element return means Tri-County and United have the same columns

# intersection between all 3 
common_columns = columns_blue.intersection(columns_tric, columns_united)
print("Columns shared among all dataframes:", common_columns)
print(len(common_columns))

Columns that Blueridge and Tri-County don't share: {'CustomersAffected', 'oID', 'OutageStatus'}
Columns that Blueridge and United don't share: {'CustomersAffected', 'zip', 'oID', 'OutageStatus'}
Columns that Tri-County and United don't share: {'zip'}
Columns shared among all dataframes: {'PriorityCustomers', 'OutageRecID', 'OutageStartTime', 'District', 'MapLocation', 'EMC', 'OutagedPhase', 'CostEstimate', 'Verified', 'CustomersServed', 'OutageEndTime', 'OutagePolygon', 'EstimatedTime', 'SuspectedCause', 'CrewResponsible', 'Status', 'CustomersOutInitially', 'Actions', 'Substation', 'timestamp', 'UplineElement', 'Notes', 'CustomerResponsible', 'OutageLocation', 'VerifiedCause', 'TroubledElement', 'Feeder', 'CustomersOutNow', 'OutageName', 'CustomersRestored', 'CrewDispatched', 'Dispatcher', 'ClosingDispatcher'}
33


After checking, we see that: 
- Blueridge has CustomersAffected and OutageStatus 
- Tri-County and United only have oID.
- Blueridge and Tri-County only have zip


We should check if all the columns including the different columns are not null so they can be used.

In [54]:
for df in dataframes:
    name = df['EMC'].iloc[0]
    print(name)
    print(df.info())

Blueridge Mountain EMC
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42513 entries, 0 to 42512
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   OutageLocation         42513 non-null  object 
 1   OutagePolygon          42513 non-null  object 
 2   OutageRecID            42513 non-null  object 
 3   OutageName             42506 non-null  object 
 4   OutageStartTime        42513 non-null  object 
 5   OutageEndTime          141 non-null    object 
 6   CustomersRestored      42513 non-null  int64  
 7   CustomersOutNow        42513 non-null  int64  
 8   Verified               42513 non-null  bool   
 9   CrewDispatched         42513 non-null  bool   
 10  CustomerResponsible    0 non-null      float64
 11  EstimatedTime          0 non-null      float64
 12  TroubledElement        0 non-null      float64
 13  Substation             0 non-null      float64
 14  Feeder                 0 non-nu

We see that many of the columns have no non-null columns or columns with VERY few non-null values.

Let's go through them and figure out which columns are usable based on having a substantial amount of non-null values.

In [55]:
non_null_columns_blue = None
non_null_columns_tric = None
non_null_columns_united = None

for df in dataframes:
    name = df['EMC'].iloc[0]
    print(name)
    # print(df.info())

    non_null_columns = df.count() > len(df) * 0.0001 # more than 0.0001% of len has nonnull values
    selected_columns = non_null_columns[non_null_columns].index.tolist()
    
    if "Blueridge" in name:
        non_null_columns_blue = set(selected_columns)
    elif "Tri-Countt" in name:
        non_null_columns_tric = set(selected_columns)
    else: # "United" in name:
        non_null_columns_united = set(selected_columns)
    print(f"{len(selected_columns)} columns with more than an inconsequential # of non-null values:", selected_columns)

    print()

diff_nonnull_blue_tric = non_null_columns_blue.symmetric_difference(non_null_columns_tric)
diff_nonnull_blue_united = non_null_columns_blue.symmetric_difference(non_null_columns_united)
diff_nonnull_tric_united = non_null_columns_tric.symmetric_difference(non_null_columns_united)

print("Nonnull columns that Blueridge and Tri-County don't share: ", diff_nonnull_blue_tric)
print("Nonnull columns that Blueridge and United don't share: ", diff_nonnull_blue_united)
print("Nonnull columns that Tri-County and United don't share: ", diff_nonnull_tric_united)

common_nonnull_columns = non_null_columns_blue.intersection(non_null_columns_tric, non_null_columns_united)
print("Non-null Columns shared among all dataframes:", common_nonnull_columns)
print(len(common_nonnull_columns))

Blueridge Mountain EMC
12 columns with more than an inconsequential # of non-null values: ['OutageLocation', 'OutagePolygon', 'OutageRecID', 'OutageName', 'OutageStartTime', 'OutageEndTime', 'CustomersRestored', 'CustomersOutNow', 'Verified', 'CrewDispatched', 'EMC', 'timestamp']

Tri-Countt EMC
13 columns with more than an inconsequential # of non-null values: ['OutageLocation', 'OutagePolygon', 'OutageRecID', 'OutageName', 'OutageStartTime', 'OutageEndTime', 'CustomersRestored', 'CustomersOutNow', 'Verified', 'CrewDispatched', 'oID', 'EMC', 'timestamp']

United Coop Services, Inc.
14 columns with more than an inconsequential # of non-null values: ['OutageLocation', 'OutagePolygon', 'OutageRecID', 'OutageName', 'OutageStartTime', 'OutageEndTime', 'CustomersRestored', 'CustomersOutNow', 'Verified', 'CrewDispatched', 'EstimatedTime', 'oID', 'timestamp', 'EMC']

Nonnull columns that Blueridge and Tri-County don't share:  {'oID'}
Nonnull columns that Blueridge and United don't share:  {'o

Let's check the extraneous non-null columns: 
- oID (for Tri-County and United) 
- EstimatedTime (shared across all 3 but only non-zero for United)

In [56]:
# tri-county
are_identical_tric = (tric_df['oID'] == tric_df['OutageRecID']).all()
# print(len(tric_df[tric_df['oID'] != tric_df['OutageRecID']])) # = 0
are_identical_united = (united_df['oID'] == united_df['OutageRecID']).all()

print("Is Tri-County's 'oID' and 'OutageRecID' identical?:", are_identical_tric)
print("Is United's 'oID' and 'OutageRecID' identical?:", are_identical_united)


Is Tri-County's 'oID' and 'OutageRecID' identical?: True
Is United's 'oID' and 'OutageRecID' identical?: True


In [57]:
united_nonnull_estimate = united_df[united_df['EstimatedTime'].notna()]
# display(united_nonnull_estimate)
estimate_only = united_nonnull_estimate[['OutageStartTime','EstimatedTime','timestamp']]
display(estimate_only)

print("Ratio of non-null EstimatedTime's:", len(estimate_only)/len(united_df))
# # Can also do through the code below
# non_null_estimatedtime_values = united_df.loc[united_df['EstimatedTime'].notna(), 'EstimatedTime']
# display(non_null_estimatedtime_values)

,OutageStartTime,EstimatedTime,timestamp
0,2023-04-06T00:00:52-05:00,2023-04-06T02:00:57-05:00,04-06-2023 05:05:28
1,2023-04-13T13:48:28-05:00,2023-04-13T15:49:42-05:00,04-13-2023 19:38:15
2,2023-04-13T13:48:28-05:00,2023-04-13T15:49:42-05:00,04-13-2023 19:50:28
3,2023-04-13T13:48:28-05:00,2023-04-13T15:49:42-05:00,04-13-2023 20:05:29
4,2023-04-13T13:48:28-05:00,2023-04-13T15:49:42-05:00,04-13-2023 20:20:28
...,...,...,...
33476,2023-09-08T23:15:26-05:00,2023-09-09T01:18:02-05:00,09-09-2023 06:05:29
33478,2023-09-08T23:15:24-05:00,2023-09-09T01:18:43-05:00,09-09-2023 06:05:29
33479,2023-09-08T23:16:57-05:00,2023-09-09T01:18:43-05:00,09-09-2023 06:05:29
33482,2023-09-08T23:18:25-05:00,2023-09-09T01:18:25-05:00,09-09-2023 06:05:29


Ratio of non-null EstimatedTime's: 0.28756114605171207


What we end up getting is these 14 non-null columns between the 3 datasets:
- 'OutagePolygon'
- 'OutageLocation'
- 'OutageEndTime' (though very few?)
- 'CrewDispatched' 
- 'OutageName'
- 'OutageStartTime'
- 'CustomersOutNow'
- 'CustomersRestored'
- 'OutageRecID' (interchangable with oID for Tri-County and United)
- 'EMC'
- 'Verified'
- 'timestamp'
- 'Estimated Time' (for United but a maybe)

As for not shared:
- United has a significant number of non-null time values in 'EstimatedTime' (~23% of the entire df) so 'EstimatedTime' may be used
- United and Tri-County's oID is identitical to their OutageRecID so oID is not needed.

These columns can be used for our data-engineering and any other column not mentioned in the above will not be used.

For example:
- CustomersAffected, customersServed, OutageStatus, and zip will NOT be used unfortunately

### Checking whether to use OutageName or OutageRecID

In [58]:
for dataframe in dataframes:
    print(f"{dataframe.EMC.iloc[0]}:")
    print(f"Unique OutageNames: {len(dataframe['OutageName'].unique())}")
    print(f"Unqiue OutageRecID's: {len(dataframe['OutageRecID'].unique())}")
    print(f"Unique locations: {len(dataframe['OutageLocation'].unique())}")
    print()

Blueridge Mountain EMC:
Unique OutageNames: 2422
Unqiue OutageRecID's: 5547
Unique locations: 4585

Tri-Countt EMC:
Unique OutageNames: 2204
Unqiue OutageRecID's: 2029
Unique locations: 14159

United Coop Services, Inc.:
Unique OutageNames: 8099
Unqiue OutageRecID's: 8099
Unique locations: 6831



We will check for unique locations for both Blueridge and Tri-County and see for each location how many unique outages names and how many unique outage rec id's they have.

In [59]:
df = blue_df #tric_df also problematic

for df in dataframes:
    print(f"{df.EMC.iloc[0]}:")
    result_df = pd.DataFrame()
    grouped = df.groupby('OutageLocation')
    unique_name = grouped['OutageName'].nunique()
    unique_id = grouped['OutageRecID'].nunique()
    result_df['nunique_outageName'] = unique_name
    result_df['nunique_outageID'] = unique_id
    display(result_df)
    print(f"All of the # of unique outageID's per unique location: {result_df['nunique_outageID'].unique()}")
    print(f"All of the # of unique outage Name's per unique location: {result_df['nunique_outageName'].unique()}")
    print(f"{len(result_df[result_df['nunique_outageID'] > 1])} unique locations that have multiple unique outage id's")
    print(f"{len(result_df[result_df['nunique_outageName'] > 1])} unique locations that have multiple unique outage names")
    print()


# df = tric_df
# print("Tri-County")
# result_df = pd.DataFrame()
# grouped = df.groupby('OutageLocation')
# unique_name = grouped['OutageName'].nunique()
# unique_id = grouped['OutageRecID'].nunique()
# result_df['nunique_outageName'] = unique_name
# result_df['nunique_outageID'] = unique_id
# display(result_df)
# print(result_df['nunique_outageID'].unique())
# print(result_df['nunique_outageName'].unique())
# print(f"{len(result_df[result_df['nunique_outageID'] > 1])} unique locations that have multiple unique outage id's")
# print(f"{len(result_df[result_df['nunique_outageName'] > 1])} unique locations that have multiple unique outage names")



Blueridge Mountain EMC:


,nunique_outageName,nunique_outageID
OutageLocation,,
"{'X': -83.55376497091625, 'Y': 34.976920325410155}",1,1
"{'X': -83.55427681158751, 'Y': 34.98685693073855}",3,3
"{'X': -83.5545616606201, 'Y': 34.98610822813316}",1,3
"{'X': -83.55949360666666, 'Y': 34.964220596666244}",1,1
"{'X': -83.59381559623294, 'Y': 34.96443222225935}",2,2
...,...,...
"{'X': -84.31324050319957, 'Y': 35.01238375064682}",1,1
"{'X': -84.31452431081752, 'Y': 35.02650600252049}",1,1
"{'X': -84.31555164722663, 'Y': 35.0109696437034}",1,1


All of the # of unique outageID's per unique location: [ 1  3  2  6  8  4  5  7 11 14]
All of the # of unique outage Name's per unique location: [1 3 2 4 5 6 7 8]
1151 unique locations that have multiple unique outage id's
1090 unique locations that have multiple unique outage names

Tri-Countt EMC:


,nunique_outageName,nunique_outageID
OutageLocation,,
"{'X': -83.1618577620116, 'Y': 33.31732312710027}",1,1
"{'X': -83.16198547998756, 'Y': 33.28770484476284}",1,1
"{'X': -83.16273802179892, 'Y': 33.316114391795736}",1,1
"{'X': -83.16332750702497, 'Y': 33.3158533820869}",1,1
"{'X': -83.16444388181446, 'Y': 33.31440853178022}",1,1
...,...,...
"{'X': -83.78684074580796, 'Y': 33.108915023210926}",1,1
"{'X': -83.78731257081225, 'Y': 33.10844319820663}",1,1
"{'X': -83.7876936758157, 'Y': 33.108062093203166}",1,1


All of the # of unique outageID's per unique location: [1]
All of the # of unique outage Name's per unique location: [1]
0 unique locations that have multiple unique outage id's
0 unique locations that have multiple unique outage names

United Coop Services, Inc.:


,nunique_outageName,nunique_outageID
OutageLocation,,
"{'X': -97.02728148396736, 'Y': 32.39245633106151}",1,1
"{'X': -97.03013457212917, 'Y': 32.3944700818393}",1,1
"{'X': -97.03064039306142, 'Y': 32.397218421212855}",1,1
"{'X': -97.03110422450314, 'Y': 32.39481347282441}",1,1
"{'X': -97.03112514997952, 'Y': 32.396279293284266}",1,1
...,...,...
"{'X': -98.78630466965889, 'Y': 32.870325056259304}",1,1
"{'X': -98.78656320471063, 'Y': 32.9362946843826}",1,1
"{'X': -98.80166346081663, 'Y': 32.89792796949177}",1,1


All of the # of unique outageID's per unique location: [ 1  2  3  4  5  6  8 12  7 28 11 10]
All of the # of unique outage Name's per unique location: [ 1  2  3  4  5  6  8 12  7 28 11 10]
1327 unique locations that have multiple unique outage id's
1327 unique locations that have multiple unique outage names



As we can see, location does not seem to be a reliable metric to group by as (except for Tri-Countt) there are multiple unique outageID's and outageName's for each unique location.


Check for each unique name how many unique rec id's there are

In [60]:
# group by unique name
for df in dataframes:
    print(f"{df.EMC.iloc[0]}:")
    result_df = pd.DataFrame()
    grouped = df.groupby('OutageName')
    unique_id = grouped['OutageRecID'].nunique()
    result_df['nunique_outageID'] = unique_id
    display(result_df)
    print(f"All of the # of unique outageID's per unique name: {result_df['nunique_outageID'].unique()}")
    print(f"{len(result_df[result_df['nunique_outageID'] > 1])} unique outageName's that have multiple unique outage id's ({len(result_df[result_df['nunique_outageID'] > 1])/len(result_df)})")
    print()


Blueridge Mountain EMC:


,nunique_outageID
OutageName,
472-581 IVY KNOB LN,1
CARLA CV,1
CHEROKEE MOUNTAIN TRL,1
DOUCETTE CIR,1
E VINEYARD CREEK,1
...,...
ZION RD/LUTHER CAIN RD,1
aska cphase,1
dunkin donuts,1


All of the # of unique outageID's per unique name: [  1   2   5   4   3   7   6  24  21  66  20  70  13   9  67  86 236 110
 259   8  80  39  60 179 131 160  46 145 128  81  54 279  72  48 204  56
 171  25 138 129  38 394  59]
451 unique outageName's that have multiple unique outage id's (0.1862866584056175)

Tri-Countt EMC:


,nunique_outageID
OutageName,
14-156 1/2-3 10935 YHwy 18 E,1
0112_REC,1
0112_REC CLINTON CKT 1,1
0242_REC,1
0252_REC,1
...,...
TEXAS CHAPEL,1
THOMAS DR,1
UPPER RIVER/COULTER LAKE,1


All of the # of unique outageID's per unique name: [ 1  2  3  5  6  4  7 13]
314 unique outageName's that have multiple unique outage id's (0.1424682395644283)

United Coop Services, Inc.:


,nunique_outageID
OutageName,
2023-04-06-0152,1
2023-04-13-0307,1
2023-04-13-0308,1
2023-04-13-0309,1
2023-04-13-0315,1
...,...
2024-01-04-0203,1
2024-01-04-0206,1
2024-01-04-0208,1


All of the # of unique outageID's per unique name: [1]
0 unique outageName's that have multiple unique outage id's (0.0)



Checking for each unique outage Rec ID how many outageName's there are

In [61]:
multiple_outageName_list = []
for df in dataframes:
    print(f"{df.EMC.iloc[0]}:")
    result_df = pd.DataFrame()
    grouped = df.groupby('OutageRecID')
    unique_name = grouped['OutageName'].nunique()
    result_df['nunique_outageName'] = unique_name
    display(result_df)
    print(f"All of the # of unique outage name's per unique outagerecID: {result_df['nunique_outageName'].unique()}")
    print(f"{len(result_df[result_df['nunique_outageName'] > 1])} unique outagerecID that have multiple unique outage names ({len(result_df[result_df['nunique_outageName'] > 1])/len(result_df)})")
    print()


Blueridge Mountain EMC:


,nunique_outageName
OutageRecID,
2023-03-15-0967,1
2023-04-14-0596,1
2023-04-14-0597,1
2023-04-14-0598,1
2023-04-14-0599,1
...,...
2024-01-17-0641,1
2024-01-17-0642,1
2024-01-17-0648,1


All of the # of unique outage name's per unique outagerecID: [1 2 3 4]
1298 unique outagerecID that have multiple unique outage names (0.23400036055525508)

Tri-Countt EMC:


,nunique_outageName
OutageRecID,
2023-03-18-0055,1
2023-04-14-0137,1
2023-04-15-0138,1
2023-04-15-0146,1
2023-04-15-0154,1
...,...
2024-01-17-0384,1
2024-01-17-0385,1
2024-01-17-0386,1


All of the # of unique outage name's per unique outagerecID: [1 2 3 4]


555 unique outagerecID that have multiple unique outage names (0.2735337604731395)

United Coop Services, Inc.:


,nunique_outageName
OutageRecID,
2023-04-06-0152,1
2023-04-13-0307,1
2023-04-13-0308,1
2023-04-13-0309,1
2023-04-13-0315,1
...,...
2024-01-04-0203,1
2024-01-04-0206,1
2024-01-04-0208,1


All of the # of unique outage name's per unique outagerecID: [1]
0 unique outagerecID that have multiple unique outage names (0.0)



As we can see, we see that there are far fewer unique outageName's per unique outageID than there are unique outageID's per unique outageName.

Up to dozens of unique outageID's per unique outageName VS. max 4 unique outageName's per unique outageID.

From this, I will use 'outageRecID' as the metric to define an outage.

#### Let's transform the dataset so we can compare the timemetrics to each other more consistently

- Converting times
- Dropping null columns (for observation purposes, in the final pipeline.py, desired columns will be chosen and undesired ones will be ignored in "compute_metrics)

In [62]:
def test_transform(dataframe):
    # helper function to truncate the seconds value of the timestamp
    def reformat_starttime(startTime): 
        # format: 2023-03-15T21:51:54-04:00
        reformatted = startTime
        if pd.notna(startTime) and "." in startTime: # if not NaN and there is a decimal (assuming only seconds can have decimals)
            # Find the index of the first period
            first_period_index = startTime.find('.')

            # Find the index of the first hyphen after the first period
            first_hyphen_after_period_index = startTime.find('-', first_period_index)
            
            # essentially removing the decimal part of the seconds 
            reformatted = startTime[:first_period_index] + startTime[first_hyphen_after_period_index:]

        return reformatted
    
    # def get_zipcode(df, geolocator, lat_field, long_field): # using geopy
    #     location = geolocator.reverse((df[lat_field], df[long_field]))
    #     zipcode = None
    #     try:
    #         zipcode = location.raw['address']['postcode']
    #     except Exception as e:
    #         zipcode = pd.NA
    #     return zipcode
    
    def get_zipcode(df, long_field, lat_field): # using arcgis package
        location = reverse_geocode((Geometry({"x":float(df[long_field]), "y":float(df[lat_field]), "spatialReference":{"wkid": 4326}})))
        return location['address']['Postal']

    

    try:
        # Convert timestamps
        eastern = tz.gettz('US/Eastern')
        utc = tz.gettz('UTC')
        # truncated_time = dataframe['timestamp'].str[:19]
        dataframe['OutageStartTime'] = dataframe['OutageStartTime'].apply(reformat_starttime)
        dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'], utc=True).dt.tz_convert(eastern)
        dataframe['OutageStartTime'] = pd.to_datetime(dataframe['OutageStartTime'], format='mixed', utc=True).dt.tz_convert(eastern)
        dataframe['OutageEndTime'] =pd.to_datetime(dataframe['OutageEndTime'], format='mixed', utc=True).dt.tz_convert(eastern)

        # extract lat and long
        dataframe['OutageLocation'] = dataframe['OutageLocation'].apply(lambda x: json.loads(x.replace("'", '"')))
        dataframe[['lat', 'long']] = dataframe['OutageLocation'].apply(lambda x: pd.Series([x['Y'], x['X']]))
        
        # gis = GIS("http://www.arcgis.com", "JK9035", "60129@GR0W3R5")
        # dataframe['zip'] = dataframe.apply(get_zipcode, lat_field = 'lat', long_field = 'long', axis=1)

        # dropping columns
        dataframe.drop(columns=['OutageLocation', 'OutagePolygon', 'Verified', 'CrewDispatched', 'CustomerResponsible','TroubledElement','Substation','Feeder',	'OutagedPhase', 'Actions','Status','Dispatcher','ClosingDispatcher','SuspectedCause','VerifiedCause','Notes','CrewResponsible','MapLocation','PriorityCustomers','District','UplineElement','CostEstimate'], inplace=True)

        dataframe.rename(columns={
            'OutageRecID':'outage_id',
            'OutageStartTime': 'start_time',
            'CustomersOutNow':'customer_affected',
            'EMC': 'utility_provider',
            'zip': 'zipcode'
        }, inplace=True)
    except Exception as e:
        print(f"An error occurred during transformation: {e}")

In [63]:
temp_conf = {'name': 'Blueridge Mountain EMC', 'state': 'ga', 'layout': 2, 'type': 'o'} # can't use blue_conf as that will make the subsequent conf, pipeline, and df have the same ref as blue_conf
pipe_copy = GA2TX17(temp_conf, base_file_path)
pipe_copy.load_data()
df = pipe_copy._data
print("Before transform:")
display(df)

Before transform:


C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\2338607318.py:35: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  self._data = pd.read_csv(file_path)


,OutageLocation,OutagePolygon,OutageRecID,OutageName,OutageStartTime,OutageEndTime,CustomersRestored,CustomersOutNow,Verified,CrewDispatched,CustomerResponsible,EstimatedTime,TroubledElement,Substation,Feeder,OutagedPhase,Actions,Status,Dispatcher,ClosingDispatcher,CustomersOutInitially,CustomersServed,SuspectedCause,VerifiedCause,Notes,CrewResponsible,MapLocation,PriorityCustomers,District,UplineElement,CostEstimate,OutageStatus,CustomersAffected,zip,EMC,timestamp
0,"{'X': -84.09432506550709, 'Y': 35.02260640467454}","[{'X': -84.09441896987332, 'Y': 35.02289220814...",2023-03-15-0967,73 TAYLOR HENSON RD,2023-03-15T21:51:54-04:00,NaN,0,2,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,Blueridge Mountain EMC,NaN
1,"{'X': -83.89494043421992, 'Y': 34.84341149605413}","[{'X': -83.89530348315697, 'Y': 34.84351043752...",2023-04-14-0596,WALDROOP 324,2023-04-14T04:41:07-04:00,NaN,0,6,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,04-14-2023 08:54:39
2,"{'X': -83.89494043421992, 'Y': 34.84341149605413}","[{'X': -83.89530348315697, 'Y': 34.84351043752...",2023-04-14-0596,WALDROOP 324,2023-04-14T04:41:07-04:00,NaN,0,6,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,04-14-2023 09:09:40
3,"{'X': -83.89494043421992, 'Y': 34.84341149605413}","[{'X': -83.89530348315697, 'Y': 34.84351043752...",2023-04-14-0596,WALDROOP 324,2023-04-14T04:41:07-04:00,NaN,0,6,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,04-14-2023 09:24:39
4,"{'X': -83.89494043421992, 'Y': 34.84341149605413}","[{'X': -83.89530348315697, 'Y': 34.84351043752...",2023-04-14-0596,WALDROOP 324,2023-04-14T04:41:07-04:00,NaN,0,6,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,04-14-2023 09:39:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42508,"{'X': -83.74961739536889, 'Y': 35.01959154876999}","[{'X': -83.75200012067978, 'Y': 35.01917373478...",2024-01-17-0652,ELF SCHOOL RD,2024-01-17T17:34:27-05:00,NaN,0,44,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,01-17-2024 23:39:39
42509,"{'X': -83.74961739536889, 'Y': 35.01959154876999}","[{'X': -83.75200012067978, 'Y': 35.01917373478...",2024-01-17-0652,ELF SCHOOL RD,2024-01-17T17:34:27-05:00,NaN,0,44,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,01-17-2024 23:54:39
42510,"{'X': -83.74961739536889, 'Y': 35.01959154876999}","[{'X': -83.75200012067978, 'Y': 35.01917373478...",2024-01-17-0652,ELF SCHOOL RD,2024-01-17T17:34:27-05:00,NaN,0,44,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,01-18-2024 00:09:39
42511,"{'X': -83.74961739536889, 'Y': 35.01959154876999}","[{'X': -83.75200012067978, 'Y': 35.01917373478...",2024-01-17-0652,ELF SCHOOL RD,2024-01-17T17:34:27-05:00,NaN,0,44,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,01-18-2024 00:24:39


In [64]:
print("After transform:")
test_transform(df) # with the reverse geocoding, it takes 30 minutes so the reverse geocoding line in test_transform will be commented out when running the notebook
display(df)

After transform:


,outage_id,OutageName,start_time,OutageEndTime,CustomersRestored,customer_affected,EstimatedTime,CustomersOutInitially,CustomersServed,OutageStatus,CustomersAffected,zipcode,utility_provider,timestamp,lat,long
0,2023-03-15-0967,73 TAYLOR HENSON RD,2023-03-15 21:51:54-04:00,NaT,0,2,NaN,NaN,NaN,NaN,NaN,unknown,Blueridge Mountain EMC,NaT,35.022606,-84.094325
1,2023-04-14-0596,WALDROOP 324,2023-04-14 04:41:07-04:00,NaT,0,6,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2023-04-14 04:54:39-04:00,34.843411,-83.894940
2,2023-04-14-0596,WALDROOP 324,2023-04-14 04:41:07-04:00,NaT,0,6,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2023-04-14 05:09:40-04:00,34.843411,-83.894940
3,2023-04-14-0596,WALDROOP 324,2023-04-14 04:41:07-04:00,NaT,0,6,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2023-04-14 05:24:39-04:00,34.843411,-83.894940
4,2023-04-14-0596,WALDROOP 324,2023-04-14 04:41:07-04:00,NaT,0,6,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2023-04-14 05:39:40-04:00,34.843411,-83.894940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42508,2024-01-17-0652,ELF SCHOOL RD,2024-01-17 17:34:27-05:00,NaT,0,44,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2024-01-17 18:39:39-05:00,35.019592,-83.749617
42509,2024-01-17-0652,ELF SCHOOL RD,2024-01-17 17:34:27-05:00,NaT,0,44,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2024-01-17 18:54:39-05:00,35.019592,-83.749617
42510,2024-01-17-0652,ELF SCHOOL RD,2024-01-17 17:34:27-05:00,NaT,0,44,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2024-01-17 19:09:39-05:00,35.019592,-83.749617
42511,2024-01-17-0652,ELF SCHOOL RD,2024-01-17 17:34:27-05:00,NaT,0,44,NaN,NaN,NaN,NaN,NaN,NaN,Blueridge Mountain EMC,2024-01-17 19:24:39-05:00,35.019592,-83.749617


### Checking between OutageEndTime, EstimatedTime, and TimeStamp to see which one is the more rigorous metric to use for duration.


Let's first transform all the dataframes

In [65]:
for dataframe in dataframes:
    test_transform(dataframe)

For each df, see the time difference between the OutageEndTime -> OutageStartTime; LastTimestamp -> OutageStartTime; Timestamp Diff

In [66]:
def _validate(group):
    utility_provider = group['utility_provider'].iloc[-1]
    num_rows = len(group)
    num_unique_lon = len(group['long'].unique())
    num_unique_lat = len(group['lat'].unique())
    num_unique_start_times = len(group['start_time'].unique())
    earliest_start_time = group['start_time'].min()
    latest_start_time = group['start_time'].max()
    diff_in_start = latest_start_time - earliest_start_time
    end_time = group['OutageEndTime'].max()
    earliest_timestamp = group['timestamp'].min()
    latest_timestamp = group['timestamp'].max()
    duration_by_timestamp = latest_timestamp - earliest_timestamp

    seconds_rem = duration_by_timestamp.total_seconds() % 900 # 900 secs = 15 minutes
    is_timestamp_dur_div_15_min = seconds_rem <= 5 or seconds_rem >= 895 # if time is 5 seconds within being divisible by 15 minutes
    duration_by_endstart = end_time - earliest_start_time if pd.notna(end_time) else pd.NaT
    duration_by_timestampstart = latest_timestamp - earliest_start_time
    timestampstart_endstart_dur_error = abs(duration_by_timestampstart - duration_by_endstart) if pd.notna(duration_by_endstart) else pd.NaT
    timestampstart_timestamp_dur_error = abs(duration_by_timestampstart - duration_by_timestamp) if pd.notna(duration_by_timestampstart) and pd.notna(duration_by_timestamp) else pd.NaT

    return pd.Series({
        'utility_provider': utility_provider,
        'num_rows': num_rows,
        'num_unique_long': num_unique_lon,
        'num_unique_lat': num_unique_lat,
        'num_unique_start_times': num_unique_start_times,
        'earliest_start_time': earliest_start_time,
        'latest_start_time': latest_start_time,
        'diff_in_start': diff_in_start,
        'end_time': end_time,
        'earliest_timestamp': earliest_timestamp,
        'latest_timestamp': latest_timestamp, 
        'duration_by_timestamp': duration_by_timestamp,
        'is_timestamp_dur_div_15_min': is_timestamp_dur_div_15_min,
        'duration_by_endstart': duration_by_endstart,
        'duration_by_timestampstart': duration_by_timestampstart,
        'timestampstart_endstart_dur_error': timestampstart_endstart_dur_error,
        'timestampstart_timestamp_dur_error': timestampstart_timestamp_dur_error
    })


In [67]:
# testing on a transformed df
test_df = blue_df
val_test = test_df.groupby("outage_id").apply(_validate)
display(val_test)

C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\3030073376.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_test = test_df.groupby("outage_id").apply(_validate)


,utility_provider,num_rows,num_unique_long,num_unique_lat,num_unique_start_times,earliest_start_time,latest_start_time,diff_in_start,end_time,earliest_timestamp,latest_timestamp,duration_by_timestamp,is_timestamp_dur_div_15_min,duration_by_endstart,duration_by_timestampstart,timestampstart_endstart_dur_error,timestampstart_timestamp_dur_error
outage_id,,,,,,,,,,,,,,,,,
2023-03-15-0967,Blueridge Mountain EMC,1,1,1,1,2023-03-15 21:51:54-04:00,2023-03-15 21:51:54-04:00,0 days,NaT,NaT,NaT,NaT,False,NaT,NaT,NaT,NaT
2023-04-14-0596,Blueridge Mountain EMC,4,1,1,1,2023-04-14 04:41:07-04:00,2023-04-14 04:41:07-04:00,0 days,NaT,2023-04-14 04:54:39-04:00,2023-04-14 05:39:40-04:00,0 days 00:45:01,True,NaT,0 days 00:58:33,NaT,0 days 00:13:32
2023-04-14-0597,Blueridge Mountain EMC,4,1,1,1,2023-04-14 07:31:22-04:00,2023-04-14 07:31:22-04:00,0 days,NaT,2023-04-14 07:39:40-04:00,2023-04-14 08:24:39-04:00,0 days 00:44:59,True,NaT,0 days 00:53:17,NaT,0 days 00:08:18
2023-04-14-0598,Blueridge Mountain EMC,1,1,1,1,2023-04-14 08:53:26-04:00,2023-04-14 08:53:26-04:00,0 days,NaT,2023-04-14 08:54:40-04:00,2023-04-14 08:54:40-04:00,0 days 00:00:00,True,NaT,0 days 00:01:14,NaT,0 days 00:01:14
2023-04-14-0599,Blueridge Mountain EMC,2,1,1,1,2023-04-14 10:17:54-04:00,2023-04-14 10:17:54-04:00,0 days,NaT,2023-04-14 10:24:39-04:00,2023-04-14 10:39:39-04:00,0 days 00:15:00,True,NaT,0 days 00:21:45,NaT,0 days 00:06:45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-17-0641,Blueridge Mountain EMC,1,1,1,1,2024-01-17 09:51:20-05:00,2024-01-17 09:51:20-05:00,0 days,NaT,2024-01-17 09:54:40-05:00,2024-01-17 09:54:40-05:00,0 days 00:00:00,True,NaT,0 days 00:03:20,NaT,0 days 00:03:20
2024-01-17-0642,Blueridge Mountain EMC,8,1,1,1,2024-01-17 09:58:55-05:00,2024-01-17 09:58:55-05:00,0 days,NaT,2024-01-17 10:09:40-05:00,2024-01-17 11:54:40-05:00,0 days 01:45:00,True,NaT,0 days 01:55:45,NaT,0 days 00:10:45
2024-01-17-0648,Blueridge Mountain EMC,1,1,1,1,2024-01-17 15:01:18-05:00,2024-01-17 15:01:18-05:00,0 days,NaT,2024-01-17 15:09:39-05:00,2024-01-17 15:09:39-05:00,0 days 00:00:00,True,NaT,0 days 00:08:21,NaT,0 days 00:08:21


In [68]:
def summary_stats(validated):
    non_nat_values = validated['timestampstart_endstart_dur_error'][~validated['timestampstart_endstart_dur_error'].isna()]

    subdf = {
        "Provider": validated['utility_provider'].iloc[0],
        "Num Unique Outages": len(validated),
        "Avg Num Rows": validated['num_rows'].mean(),
        "Avg Num Unique Longitude": validated['num_unique_long'].mean(),
        "Avg Num Unique Latitude": validated['num_unique_lat'].mean(),
        "Avg Num Unique start_time": validated['num_unique_start_times'].mean(),
        "Median Num Unique start_time": validated['num_unique_start_times'].median(),
        "Max Num Unique start_time": validated['num_unique_start_times'].max(),
        "Proportion of Outages with Multiple start_time": f"{len(validated[validated['num_unique_start_times']>1])/len(validated)} ({len(validated[validated['num_unique_start_times']>1])} outages)",
        "Avg Diff in start_time": validated['diff_in_start'].mean(),
        "Median Diff in start_time": validated['diff_in_start'].median(),
        "Max Diff in start_time": validated['diff_in_start'].max(), 
        "Num of Outages with an non-NA Endtime": f"{(pd.notna(validated['end_time'])).sum()} ({(pd.notna(validated['end_time'])).sum()/len(validated)})",
        "is_timestamp_dur_div_15_min freq (with 5 sec of error)": len(validated[validated['is_timestamp_dur_div_15_min']]) / len(validated),
        "Avg duration_by_timestamp": validated['duration_by_timestamp'].mean(),
        "Avg duration_by_timestampstart": validated['duration_by_timestampstart'].mean(),
        "Avg timestampstart_timestamp_dur_error": validated['timestampstart_timestamp_dur_error'].mean(),  
        "Median timestampstart_timestamp_dur_error": validated['timestampstart_timestamp_dur_error'].median(),
        "Max timestampstart_timestamp_dur_error": validated['timestampstart_timestamp_dur_error'].max(),
        "Avg timestampstart_endstart_dur_error": non_nat_values.mean(),
        "Proportion of timestampstart dur within 30 min of timestamp dur": f"{len(validated[abs(validated['timestampstart_timestamp_dur_error']) < timedelta(minutes=30)])/len(validated)} ({len(validated[abs(validated['timestampstart_timestamp_dur_error']) < timedelta(minutes=30)])})"
    }
    return subdf

In [69]:
summary_data = []
for df in dataframes:
    validated = df.groupby('outage_id').apply(_validate, include_groups=False).reset_index()
        # validated is the validated rows for a provider where each row is a unique outage_id
    summary_data.append(summary_stats(validated))
display(pd.DataFrame(summary_data))

,Provider,Num Unique Outages,Avg Num Rows,Avg Num Unique Longitude,Avg Num Unique Latitude,Avg Num Unique start_time,Median Num Unique start_time,Max Num Unique start_time,Proportion of Outages with Multiple start_time,Avg Diff in start_time,Median Diff in start_time,Max Diff in start_time,Num of Outages with an non-NA Endtime,is_timestamp_dur_div_15_min freq (with 5 sec of error),Avg duration_by_timestamp,Avg duration_by_timestampstart,Avg timestampstart_timestamp_dur_error,Median timestampstart_timestamp_dur_error,Max timestampstart_timestamp_dur_error,Avg timestampstart_endstart_dur_error,Proportion of timestampstart dur within 30 min of timestamp dur
0,Blueridge Mountain EMC,5547,7.664143,1.163512,1.163512,1.009915,1.0,3,0.009554714259960339 (53 outages),0 days 00:00:17.284478096,0 days,0 days 05:30:11,21 (0.0037858301784748512),0.999099,0 days 01:40:34.970248827,0 days 01:55:20.212585647,0 days 00:14:45.242336819,0 days 00:08:05,0 days 21:31:11,0 days 01:51:33.904761904,0.956372814133766 (5305)
1,Tri-Countt EMC,2029,6.978314,6.978314,6.977329,1.016757,1.0,3,0.016264169541646133 (33 outages),0 days 00:00:21.245441103,0 days,0 days 06:23:32,19 (0.009364218827008379),0.997043,0 days 01:30:45.693786982,0 days 01:43:39.403353057,0 days 00:12:53.709566074,0 days 00:08:00.500000,0 days 21:13:14,0 days 10:40:54.156842105,0.9674716609167078 (1963)
2,"United Coop Services, Inc.",8099,6.360785,1.072478,1.072602,1.006421,1.0,3,0.0062970737128040495 (51 outages),0 days 00:00:04.632547228,0 days,0 days 01:53:50,40 (0.004938881343375725),0.999753,0 days 01:20:38.825287072,0 days 04:02:34.941597728,0 days 02:41:56.116310655,0 days 00:11:02,27 days 04:28:03,0 days 01:18:09.097575,0.9217187307074948 (7465)


From the extensive summary table, there are a few things to note.
- Virtually all of the outage's timestamp diff are in 15 minute intervals (all three providers have a 99%+ proportion) meaning duration by timestamp may not be the most precise
- Most of the unique outages have a single start_time
- When trying to see if using OutageEndTime is valid, not only are there less than 50 with valid outageEndtimes out of thousands of outages, but of those valid ones, the error between (timestamp - start) dur and (endtime - start) dur averages at more than 1 hour. Thus, using endtime is dubious.
- For (timestamp - start) (aka timestampstart). 
    - We see when comparing the error between (timestamp - start) dur and timestamp dur (our most consistent metric) that over 90% of each provider has the (timestamp - start) dur within 30 minutes of the timestamp dur. 
    - However, there are outliers that need to be considered within United Coop Services
    - When we compare the average duration by timestamp and the average duration by (timestamp-start), both averages are within 2 hours and within 15 minutes of each other for Blueridge and Tri-Countt.
    - However, for United, the average duration by timestamp is also within two hours but the average duration by (timestamp-start) is over 4 hours. 
        - This is indicative of timestamps not being always right after the outage starts and could be many days or even weeks after the outage starts (which an outage weeks long should not be possible) and perhaps after the outage ends
        - The difference in timestamps seems to be consistent with how long an outage should be while (timestamp-start) would only be reliable if the timestamp was right after the outage starts.    

Thus, the duration will be calculated by timestamp differences using the first start_time as the start.

 



In [70]:
# Finding the outlier in United Coop Services Inc.
test_df = united_df
val_test = test_df.groupby("outage_id").apply(_validate)
display(val_test[val_test['timestampstart_timestamp_dur_error'] > timedelta(days=15)])


C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\2732357286.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_test = test_df.groupby("outage_id").apply(_validate)


,utility_provider,num_rows,num_unique_long,num_unique_lat,num_unique_start_times,earliest_start_time,latest_start_time,diff_in_start,end_time,earliest_timestamp,latest_timestamp,duration_by_timestamp,is_timestamp_dur_div_15_min,duration_by_endstart,duration_by_timestampstart,timestampstart_endstart_dur_error,timestampstart_timestamp_dur_error
outage_id,,,,,,,,,,,,,,,,,
2023-05-20-4300,"United Coop Services, Inc.",2,1,1,1,2023-05-04 22:09:40-04:00,2023-05-04 22:09:40-04:00,0 days,NaT,2023-05-20 12:35:28-04:00,2023-05-20 12:50:28-04:00,0 days 00:15:00,True,NaT,15 days 14:40:48,NaT,15 days 14:25:48
2023-07-05-0691,"United Coop Services, Inc.",1,1,1,1,2023-06-10 18:20:33-04:00,2023-06-10 18:20:33-04:00,0 days,NaT,2023-07-05 18:50:28-04:00,2023-07-05 18:50:28-04:00,0 days 00:00:00,True,NaT,25 days 00:29:55,NaT,25 days 00:29:55
2023-08-31-0804,"United Coop Services, Inc.",1,1,1,1,2023-08-10 17:54:37-04:00,2023-08-10 17:54:37-04:00,0 days,NaT,2023-08-31 15:50:28-04:00,2023-08-31 15:50:28-04:00,0 days 00:00:00,True,NaT,20 days 21:55:51,NaT,20 days 21:55:51
2023-09-26-2903,"United Coop Services, Inc.",1,1,1,1,2023-09-05 10:03:17-04:00,2023-09-05 10:03:17-04:00,0 days,NaT,2023-09-26 12:35:28-04:00,2023-09-26 12:35:28-04:00,0 days 00:00:00,True,NaT,21 days 02:32:11,NaT,21 days 02:32:11
2023-09-26-2905,"United Coop Services, Inc.",1,1,1,1,2023-09-11 09:11:48-04:00,2023-09-11 09:11:48-04:00,0 days,NaT,2023-09-26 12:50:28-04:00,2023-09-26 12:50:28-04:00,0 days 00:00:00,True,NaT,15 days 03:38:40,NaT,15 days 03:38:40
2023-10-03-0032,"United Coop Services, Inc.",1,1,1,1,2023-09-16 21:01:01-04:00,2023-09-16 21:01:01-04:00,0 days,NaT,2023-10-03 12:20:28-04:00,2023-10-03 12:20:28-04:00,0 days 00:00:00,True,NaT,16 days 15:19:27,NaT,16 days 15:19:27
2023-10-27-0812,"United Coop Services, Inc.",1,1,1,1,2023-10-07 08:39:52-04:00,2023-10-07 08:39:52-04:00,0 days,NaT,2023-10-27 16:35:28-04:00,2023-10-27 16:35:28-04:00,0 days 00:00:00,True,NaT,20 days 07:55:36,NaT,20 days 07:55:36
2023-12-07-0229,"United Coop Services, Inc.",2,2,2,1,2023-11-09 23:07:25-05:00,2023-11-09 23:07:25-05:00,0 days,NaT,2023-12-07 03:20:28-05:00,2023-12-07 03:35:28-05:00,0 days 00:15:00,True,NaT,27 days 04:28:03,NaT,27 days 04:13:03
2023-12-07-0230,"United Coop Services, Inc.",1,1,1,1,2023-11-09 23:50:51-05:00,2023-11-09 23:50:51-05:00,0 days,NaT,2023-12-07 03:35:28-05:00,2023-12-07 03:35:28-05:00,0 days 00:00:00,True,NaT,27 days 03:44:37,NaT,27 days 03:44:37


# Running the Pipelines

In [77]:
# 10 minute run time
for provider in config['providers']:
    pipeline = GA2TX17(provider, base_file_path)
    pipeline.standardize()
    display(pipeline._data)

C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\3760018896.py:35: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  self._data = pd.read_csv(file_path)
C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\3760018896.py:146: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)


,outage_id,start_time,end_time,lat,long,zipcode,county_name,county_fips,state,utility_provider,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time
0,2023-03-15-0967,2023-03-15 21:51:54-04:00,NaT,35.02,-84.09,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,NaT,NaT,2.0,NaT
1,2023-04-14-0596,2023-04-14 04:41:07-04:00,2023-04-14 05:26:08-04:00,34.84,-83.89,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 01:00:01,0 days 00:52:31,6.0,0 days 04:30:06
2,2023-04-14-0597,2023-04-14 07:31:22-04:00,2023-04-14 08:16:21-04:00,34.97,-84.11,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 00:59:59,0 days 00:52:29,85.0,2 days 15:43:35
3,2023-04-14-0598,2023-04-14 08:53:26-04:00,2023-04-14 08:53:26-04:00,34.97,-84.11,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 00:15:00,0 days 00:07:30,1.0,0 days 00:00:00
4,2023-04-14-0599,2023-04-14 10:17:54-04:00,2023-04-14 10:32:54-04:00,34.98,-83.82,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 00:30:00,0 days 00:22:30,6.0,0 days 01:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542,2024-01-17-0641,2024-01-17 09:51:20-05:00,2024-01-17 09:51:20-05:00,35.04,-84.15,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 00:15:00,0 days 00:07:30,1.0,0 days 00:00:00
5543,2024-01-17-0642,2024-01-17 09:58:55-05:00,2024-01-17 11:43:55-05:00,35.05,-83.79,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 02:00:00,0 days 01:52:30,3.0,0 days 05:15:00
5544,2024-01-17-0648,2024-01-17 15:01:18-05:00,2024-01-17 15:01:18-05:00,35.17,-84.12,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 00:15:00,0 days 00:07:30,1.0,0 days 00:00:00
5545,2024-01-17-0650,2024-01-17 15:45:15-05:00,2024-01-17 15:45:15-05:00,35.00,-84.30,000000,<NA>,<NA>,<NA>,Blueridge Mountain EMC,0 days 00:15:00,0 days 00:07:30,3.0,0 days 00:00:00


C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\3760018896.py:146: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)


,outage_id,start_time,end_time,lat,long,zipcode,county_name,county_fips,state,utility_provider,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time
0,2023-03-18-0055,2023-03-18 15:18:04-04:00,NaT,33.20,-83.40,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,NaT,NaT,2.0,NaT
1,2023-04-14-0137,2023-04-14 13:23:00-04:00,2023-04-14 14:08:00-04:00,32.90,-83.50,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 01:00:00,0 days 00:52:30,2.0,0 days 01:30:00
2,2023-04-15-0138,2023-04-15 05:11:37-04:00,2023-04-15 06:11:37-04:00,33.17,-83.44,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 01:15:00,0 days 01:07:30,1009.0,42 days 01:00:00
3,2023-04-15-0146,2023-04-15 06:39:54-04:00,2023-04-15 07:09:54-04:00,33.19,-83.43,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 00:45:00,0 days 00:37:30,853.0,17 days 18:30:00
4,2023-04-15-0154,2023-04-15 14:06:14-04:00,2023-04-15 14:21:13-04:00,33.13,-83.48,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 00:29:59,0 days 00:22:29,1.0,0 days 00:14:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024-01-17-0384,2024-01-17 12:12:12-05:00,2024-01-17 12:12:12-05:00,33.20,-83.42,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 00:15:00,0 days 00:07:30,2.0,0 days 00:00:00
2025,2024-01-17-0385,2024-01-17 12:18:26-05:00,2024-01-17 12:18:26-05:00,33.00,-83.66,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 00:15:00,0 days 00:07:30,1.0,0 days 00:00:00
2026,2024-01-17-0386,2024-01-17 17:21:49-05:00,2024-01-17 19:36:51-05:00,33.28,-83.23,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 02:30:02,0 days 02:22:32,1.0,0 days 02:15:02
2027,2024-01-17-0387,2024-01-17 19:22:48-05:00,2024-01-17 20:37:45-05:00,33.34,-83.51,000000,<NA>,<NA>,<NA>,Tri-Countt EMC,0 days 01:29:57,0 days 01:22:27,1.0,0 days 01:14:57


C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\3760018896.py:35: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  self._data = pd.read_csv(file_path)
C:\Users\uirja\AppData\Local\Temp\ipykernel_10460\3760018896.py:146: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)


,outage_id,start_time,end_time,lat,long,zipcode,county_name,county_fips,state,utility_provider,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time
0,2023-04-06-0152,2023-04-06 01:00:52-04:00,2023-04-06 01:00:52-04:00,32.40,-97.41,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:15:00,0 days 00:07:30,1.0,0 days 00:00:00
1,2023-04-13-0307,2023-04-13 14:48:28-04:00,2023-04-13 15:45:41-04:00,32.47,-97.12,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 01:12:13,0 days 01:04:43,2.0,0 days 01:54:26
2,2023-04-13-0308,2023-04-13 18:02:04-04:00,2023-04-13 18:02:04-04:00,32.33,-97.08,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:15:00,0 days 00:07:30,2.0,0 days 00:00:00
3,2023-04-13-0309,2023-04-13 20:40:24-04:00,2023-04-13 20:40:24-04:00,32.11,-98.44,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:15:00,0 days 00:07:30,1.0,0 days 00:00:00
4,2023-04-13-0315,2023-04-13 23:20:58-04:00,2023-04-13 23:35:58-04:00,32.33,-97.71,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:30:00,0 days 00:22:30,52.0,0 days 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8094,2024-01-04-0203,2024-01-04 20:25:49-05:00,2024-01-04 20:55:49-05:00,32.31,-98.11,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:45:00,0 days 00:37:30,1.0,0 days 00:30:00
8095,2024-01-04-0206,2024-01-04 23:08:32-05:00,2024-01-04 23:08:32-05:00,32.45,-97.55,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:15:00,0 days 00:07:30,4.0,0 days 00:00:00
8096,2024-01-04-0208,2024-01-05 00:15:11-05:00,2024-01-05 01:00:11-05:00,32.43,-97.69,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 01:00:00,0 days 00:52:30,4.0,0 days 03:00:00
8097,2024-01-04-0209,2024-01-05 00:52:11-05:00,2024-01-05 01:07:11-05:00,32.31,-98.11,000000,<NA>,<NA>,<NA>,"United Coop Services, Inc.",0 days 00:30:00,0 days 00:22:30,1.0,0 days 00:15:00
